In [1]:
# Building inverse paraphraser

In [2]:
import torch
import tqdm

In [4]:
# declaring variables
model_type = "gpt2"
model_name_or_path = "gpt2" #gpt2-large
data_dir = ""
save_steps = 500
num_epochs = 3
gradient_accumulation_steps = 2
per_gpu_train_batch_size = 5
learning_rt = 5e-5
prefix_input_type = "paraphrase_250"
extra_embedding_dim = 768 # Size of linear layer used for projecting extra embeddings.
global_dense_feature_list = None
job_id = "shakespeare_0"
specific_style_train = 0

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

print(device, n_gpus)

cpu 0


In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00


In [7]:
from transformers import (AdamW, GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, get_linear_schedule_with_warmup)

In [8]:
MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
}

In [9]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
print(config_class, model_class, tokenizer_class, sep="\n")

<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
<class 'transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer'>


In [10]:
config = config_class.from_pretrained(model_name_or_path)

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [11]:
config.extra_embedding_dim = extra_embedding_dim # don't know why we are using it; -- explore it

In [12]:
tokenizer = tokenizer_class.from_pretrained(model_name_or_path,do_lower_case = False)

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
model = model_class.from_pretrained(model_name_or_path, config=config)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
SPECIAL_TOKENS = {
    "additional_special_tokens": ["<dense-vectors>", "<tokens>", "<vercourb>", "<ARG0>", "<ARG1>", "<global-dense-vectors>"],
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
} # total 9 tokens

In [15]:
tokenizer.add_special_tokens(SPECIAL_TOKENS)

9

In [16]:
len(tokenizer) # in general 50257; but 9 are added so becomes 50266

50266

In [17]:
model.resize_token_embeddings(len(tokenizer)) # each token of size-> 1280

Embedding(50266, 768)

In [18]:
import tqdm
from torch.utils.data import Dataset
import pickle
import numpy as np

In [19]:
from collections import defaultdict

BATCH_SIZE = 128
MAX_LENGTH = 502
MAX_GPT2_LENGTH = 1000

MAX_PARAPHRASE_LENGTH = 100

BASE_CONFIG = {
    "keys": [
        {"key": "sent1_tokens", "position": 3, "tokenize": True, "metadata": False},
        {"key": "sent2_tokens", "position": 4, "tokenize": True, "metadata": False},
        {"key": "f1_score", "position": 5, "tokenize": False, "metadata": True},
        {"key": "kt_score", "position": 6, "tokenize": False, "metadata": True},
        {"key": "ed_score", "position": 7, "tokenize": False, "metadata": True},
        {"key": "langid", "position": 8, "tokenize": False, "metadata": True},
    ],
    "max_total_length": MAX_PARAPHRASE_LENGTH,
    "max_prefix_length": int(MAX_PARAPHRASE_LENGTH / 2),
    "max_suffix_length": int(MAX_PARAPHRASE_LENGTH / 2),
    "max_dense_length": 2
}

In [20]:
DATASET_CONFIG = {
"datasets/test_custom": BASE_CONFIG,
"datasets/paranmt_filtered": BASE_CONFIG,
    "datasets/shakespeare": BASE_CONFIG,
    "datasets/formality": BASE_CONFIG,
    "datasets/shakespeare_aae_tweets_bible_romantic-poetry_switchboard_coha_3_bins_lyrics_full": BASE_CONFIG,
    "datasets/aae": BASE_CONFIG,
    "datasets/bible": BASE_CONFIG,
    "datasets/poetry": BASE_CONFIG,
    "datasets/switchboard": BASE_CONFIG,
    "datasets/english_tweets": BASE_CONFIG,
    "datasets/lyrics_full": BASE_CONFIG,
    "datasets/joyce": BASE_CONFIG,
    "datasets/congress-bills": BASE_CONFIG,
    "datasets/shakespeare": BASE_CONFIG,
    "datasets/coha_3_bins_1810s-1820s": BASE_CONFIG,
    "datasets/coha_3_bins_1890s-1900s": BASE_CONFIG,
    "datasets/coha_3_bins_1990s-2000s": BASE_CONFIG
}

In [21]:
def string_to_ids(text, tokenizer):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [22]:
tokenizer.pad_token_id

50263

In [23]:
limit_examples = 10

In [25]:

def get_label_dict(data_dir):
  label_dict = {}
  with open("{}/dict.txt".format(data_dir)) as f:
      label_dict_lines = f.read().strip().split("\n")

  for i, x in enumerate(label_dict_lines):
      if x.startswith("madeupword"):
          continue
      label_dict[x.split()[0]] = i

  reverse_label_dict = {v: k for k, v in label_dict.items()}
  return label_dict, reverse_label_dict

In [72]:
def limit_styles(dataset, specific_style_train, split):
    """Limit the dataset size to a certain author."""
    # specific_style_train = [int(x) for x in specific_style_train.split(",")]

    original_dataset_size = len(dataset)
    if split in ["train", "test"] and specific_style_train != -1:
        dataset = [x for x in dataset if x["suffix_style"] == specific_style_train]
    return dataset

In [81]:
def preprocess(exp, tokenizer):
  # truncate
  max_prefix_len = BASE_CONFIG["max_prefix_length"]
  max_suffix_len = BASE_CONFIG["max_suffix_length"]


  sent1 = np.array(exp["prefix_sentence"]) # sent1_tokens
  sent2 = np.array(exp["sentence"]) # sent2_tokens

  if(len(sent1) > max_prefix_len):
    sent1 = sent1[:max_prefix_len]

  if(len(sent2) > max_suffix_len):
    sent2 = sent2[:max_suffix_len]

  # shuffle (prefix and suffix; we will not do this)

  # add padding; left padding to prefix and right padding to suffix
  count_pad_tokens_prefix = max_prefix_len - len(sent1)
  sent1 = np.pad(sent1, (count_pad_tokens_prefix, 0), constant_values = tokenizer.pad_token_id)

  # add <eos> to suffix
  sent2 = np.append(sent2, tokenizer.eos_token_id)

  count_pad_tokens_suffix = (max_suffix_len + 1) - len(sent2)
  sent2 = np.pad(sent2, (0, count_pad_tokens_suffix), constant_values = tokenizer.pad_token_id)

  # sentence to input gpt2
  sentence_to_input_gpt2 = np.concatenate([sent1, [tokenizer.bos_token_id], sent2]).astype(np.int64) # [sent1, <bos> sent2]

  # label/gt to predict
  gt = np.concatenate([
      [-100 for _ in sent1],
      [-100],
      [val if val != tokenizer.pad_token_id else -100 for val in sent2]
  ]).astype(np.int64)

  # segment
  segment = np.concatenate([
      [tokenizer.additional_special_tokens_ids[1] for _ in sent1],
      [tokenizer.additional_special_tokens_ids[2]],
      [tokenizer.additional_special_tokens_ids[2] for _ in sent2]
  ]).astype(np.int64)

  exp["prefix_sent"] = sent1
  exp["suffix_sent"] = sent2

  exp["input"] = sentence_to_input_gpt2 # sentence
  exp["label"] = gt # label
  exp["segment"] = segment # segment

  return exp

In [79]:
class InverseParaphraseDatasetText(Dataset):
  def __init__(self, tokenizer, split):
    data_dir = "/content/drive/MyDrive/IRE_Project/style_transfer_paraphrase/datasets/shakespeare"
    prefix_input_type = "paraphrase_250"
    self.config = BASE_CONFIG

    self.label_dict, self.reverse_label_dict = get_label_dict(data_dir)
    self.global_dense_features = []

    # load data
    with open(data_dir + "/train.input0.bpe") as f:
      author_input_data = f.read().strip().split("\n")

    with open(data_dir + "/train.label") as f:
      suffix_styles = f.read().strip().split("\n")

    with open(data_dir + "/train.paraphrase_250_input0.bpe") as f:
      prefix_data = f.read().strip().split("\n")

    # print(suffix_styles) # [original, modern]

    print(author_input_data[5]) # actual bpe
    print(prefix_data[5]) # after DVP bde
    print(suffix_styles[5])

    # print(True if "original" in suffix_styles[18395:] else False)
    # print(True if "modern" in suffix_styles[:18394] else False)

    # print(len(author_input_data), len(suffix_styles), len(prefix_data), sep = "\n")

    self.examples = []

    for i, (inp, suffix_style) in tqdm.tqdm(enumerate(zip(author_input_data, suffix_styles)),
                                                    total=len(author_input_data)):
      self.examples.append({
          "prefix_sentence": np.array([int(x) for x in prefix_data[i].split()], dtype=np.int32), # from DVP
          "sentence": np.array([int(x) for x in inp.split()], dtype=np.int32), # from actual
          "suffix_style": self.label_dict[suffix_style],
      })

      # limit style to specific_style_train arg
    self.examples = limit_styles(self.examples, specific_style_train, split)
    self.examples = self.examples[:limit_examples]


    for eg in self.examples:
      eg["original_style"] = eg["suffix_style"]

    # perform same preprocess as that of DVP
    self.examples = [preprocess(exp, tokenizer) for exp in self.examples]


  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    sentence = self.examples[item]["sentence"]
    label = self.examples[item]["label"]
    segment = self.examples[item]["segment"]
    suffix_style = self.examples[item]["suffix_style"]
    original_style = self.examples[item]["original_style"]
    init_context_size = 1 + BASE_CONFIG["max_prefix_length"]
    global_dense_vectors = np.zeros((1, 768), dtype=np.float32)

    # metadata = "suffix_style = {}, original_style = {}".format(
    #         suffix_style, original_style
    #     )

    return {
            "sentence": torch.tensor(sentence),
            "instance_number": item,
            "label": torch.tensor(label),
            "segment": torch.tensor(segment),
            "suffix_style": suffix_style,
            "original_style": original_style,
            "init_context_size": init_context_size,
            "global_dense_vectors": global_dense_vectors,
            # "metadata": self.examples[item].dict["metadata"],
        }

In [89]:
import tqdm

In [90]:
# load dataset
train_dataset = InverseParaphraseDatasetText(tokenizer=tokenizer, split="train")

817 6 1242 281 5508 582 13
270 338 257 1103 582 338 1693 13
original


100%|██████████| 36790/36790 [00:01<00:00, 26680.59it/s]


In [91]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange

In [92]:
# start training
train_batch_size = per_gpu_train_batch_size
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=train_batch_size)

In [93]:
# total gradient descent steps
t_total = len(train_dataloader) // gradient_accumulation_steps * num_epochs
t_total

3

In [94]:
adam_epsilon = 1e-8
warmup_steps = 0

In [95]:
# setting up optimizer and scheduler for training
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']

grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters()],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(grouped_parameters,
                lr=float(learning_rt),
                eps=adam_epsilon)

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [96]:
# total information before training
print("Num of examples- ", len(train_dataset))
print("Num of epochs- ", num_epochs)
print("Batch size- ", train_batch_size)
print("Gradient acculmulation steps- ", gradient_accumulation_steps)
print("Total optimization steps- ", t_total)

Num of examples-  10
Num of epochs-  3
Batch size-  5
Gradient acculmulation steps-  2
Total optimization steps-  3


In [97]:
model.zero_grad()

In [98]:
train_iterator = trange(int(num_epochs), desc="Epoch")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

In [99]:
max_grad_norm = 1.0
save_steps = 2
global_step = 0
loss_val = 0.0

In [100]:
import os

In [101]:
def save_model(model, tokenizer, chkpt_dir):
  if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir)

  print("Directory created")
  model.save_pretrained(chkpt_dir)
  tokenizer.save_pretrained(chkpt_dir)

  # save training arguments also

  with open(os.path.join(chkpt_dir, "global_step.txt"), "w") as f:
    f.write(str(global_step) + "\n")

  print("Model saved")

In [ ]:
for epoch in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for batch_idx, batch in enumerate(epoch_iterator):
      sentences = batch["sentence"].to(device)
      labels = batch["label"].to(device)
      segments = batch["segment"].to(device)
      global_dense_vectors = batch["global_dense_vectors"].to(device)

      # print(sentences.shape)
      model.train()

      outputs = model(
                input_ids=sentences,
                token_type_ids=segments,
                labels=labels
            )

      loss = outputs.loss
      loss = loss / gradient_accumulation_steps
      loss_val += loss.item()

      # chkpt_dir = "/content/drive/MyDrive/IRE/inversepara_shakespeare_"+str(global_step)
      # save_model(model, tokenizer, chkpt_dir)

      loss.backward()

      if (((batch_idx + 1) % gradient_accumulation_steps) == 0):
        # do gradient descent after gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        model.zero_grad()
        global_step += 1

        if (global_step % save_steps == 0):
          # save model here
          chkpt_dir = "/content/drive/MyDrive/IRE/inversepara_shakespeare_"+str(global_step)
          save_model(model, tokenizer, chkpt_dir)

      break

In [ ]:
# avg_loss_per_step = loss_val / global_step
global_step, tr_loss = global_step, loss_val / (global_step+1) # +1 only while testing

In [ ]:
# save the last state also
chkpt_dir = "/content/drive/MyDrive/IRE/inversepara_shakespeare_"+str(global_step)
save_model(model, tokenizer, chkpt_dir)

In [ ]:
# load the best DVP model

# Convert shakespeare sentences to their paraphrased version

# Convert sentences to their bpe files